In [2]:
# Inherited: `data` 
#%run "C:\Users\Asus\Desktop\Coding\API_Flask\side_BenckMark\embedding_Workspace\bench\2_Preprocess.ipynb" 

import dill
# Load session 
dill.load_session('embedded_Data.pkl') 

In [3]:
data.head(2)

,Review,Score,Sentiment,review_Embedding,semantic_Label,label_Embedding
0,"The build quality is cheap, and the plastic pa...",1,A,"[-0.012376603670418262, 0.017603429034352303, ...",Terrible,"[-0.012227306142449379, 0.005636125337332487, ..."
1,"They work, but the battery life is nowhere nea...",2,B,"[-0.012578997761011124, -0.007529035210609436,...",Mediocre,"[-0.01267366111278534, -0.0012541643809527159,..."


### `Predictions`

---
### `Result` 

#### `Setup` 

##### `Classification Report` 
* methods: "`create_Report`", "`extract_summary_metrics`" 

In [4]:
from sklearn.metrics import classification_report 

In [5]:
def create_Report(current_Prediction):
    global cosine_Report 
    
    # Get the classification report with zero_division=1 to handle division by zero
    report = classification_report(data["semantic_Label"], data[current_Prediction], zero_division=1)
    
    title_List, precision_List, recall_List, f1_List, support_List = [], [], [], [], [] 

    for line in report.split("\n")[2:-5]:  # Skip the first two lines and the last five lines of the report
        try:
            # Split the line by spaces but exclude empty splits
            values = [value for value in line.split("  ") if value]
            
            title, precision, recall, f1, support = values
            title_List.append(title.strip())
            precision_List.append(float(precision))
            recall_List.append(float(recall))
            f1_List.append(float(f1))
            support_List.append(int(support))
        except:
            pass 

    # Create dataframe from the values 
    cosine_Report = pd.DataFrame({
        'title': title_List, 
        'precision': precision_List, 
        'recall': recall_List, 
        'f1': f1_List, 
        'support': support_List
    })    
    
    return cosine_Report


In [6]:
def extract_summary_metrics(current_Prediction):
    # Get the classification report with zero_division=1 to handle division by zero
    report = classification_report(data["semantic_Label"], data[current_Prediction], zero_division=1)
    
    # Split the report by lines
    lines = report.split("\n")
    
    # Extract accuracy
    try:
        accuracy = float(lines[-2].split(":")[1].strip())
    except:
        accuracy = float('NaN')
    
    # Extract macro avg metrics
    macro_avg = lines[-4].split()
    try:
        macro_precision = float(macro_avg[2])
    except:
        macro_precision = float('NaN')
    try:
        macro_recall = float(macro_avg[3])
    except:
        macro_recall = float('NaN')
    try:
        macro_f1 = float(macro_avg[4])
    except:
        macro_f1 = float('NaN')
    try:
        macro_support = int(macro_avg[5])
    except:
        macro_support = float('NaN')
    
    # Extract weighted avg metrics
    weighted_avg = lines[-3].split()
    try:
        weighted_precision = float(weighted_avg[2])
    except:
        weighted_precision = float('NaN')
    try:
        weighted_recall = float(weighted_avg[3])
    except:
        weighted_recall = float('NaN')
    try:
        weighted_f1 = float(weighted_avg[4])
    except:
        weighted_f1 = float('NaN')
    try:
        weighted_support = int(weighted_avg[5])
    except:
        weighted_support = float('NaN')
    
    # Create a DataFrame with the extracted metrics
    summary_df = pd.DataFrame({
        'title': ['accuracy', 'macro avg', 'weighted avg'],
        'precision': [float('NaN'), macro_precision, weighted_precision],
        'recall': [float('NaN'), macro_recall, weighted_recall],
        'f1': [float('NaN'), macro_f1, weighted_f1],
        'support': [accuracy, macro_support, weighted_support]
    })
    
    return summary_df


##### `Similarity Calculations`
* method: "`label_Similarity_Score`", "`predict_1`", "`predict_2`"

In [7]:
from scipy.spatial import distance

def label_Similarity_Score(review_embedding, label_embeddings, method='Cosine'):
    scores = {}
    
    for label, embedding in label_embeddings.items():
        if method == 'Cosine':
            scores[label] = 1 - distance.cosine(review_embedding, embedding)
        elif method == 'Euclidean':
            scores[label] = 1 / (1 + distance.euclidean(review_embedding, embedding))
        elif method == 'Manhattan':
            scores[label] = 1 / (1 + distance.cityblock(review_embedding, embedding))
        elif method == 'Minkowski':
            scores[label] = 1 / (1 + distance.minkowski(review_embedding, embedding, p=3))
        elif method == 'Pearson':
            scores[label] = np.corrcoef(review_embedding, embedding)[0, 1] # Pearson is not a distance metric, so we use correlation coefficient instead 
        elif method == 'Canberra':
            scores[label] = 1 / (1 + distance.canberra(review_embedding, embedding))
       
        else:
            raise ValueError(f"Unknown method: {method}")
    
    return scores

def predict_1(similarity_scores, n=2):
    # Sort labels based on scores and get the top n labels
    sorted_labels = sorted(similarity_scores, key=similarity_scores.get, reverse=True)
    return sorted_labels[:n][0]

def predict_2(scores):
    max_index = np.argmax(scores)
    if max_index < len(current_Labels):
        return current_Labels[max_index]
    else:
        return 'External'  # or any other placeholder 

#### `Process` 

##### `Similarities & Predictions` 
* Output: "`Probabilities`" & "`Labels`" 

In [8]:
# Get Similarity_Score for each review
data["Similarity_Score_Cosine"]    = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Cosine')) 
data["Similarity_Score_Euclidean"] = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Euclidean')) 
data["Similarity_Score_Manhattan"] = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Manhattan')) 
data["Similarity_Score_Minkowski"] = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Minkowski')) 
data["Similarity_Score_Pearson"]   = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Pearson')) 
data["Similarity_Score_Canberra"]  = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Canberra')) 

# Assign top 2 labels for each review 
data["predicted_labels_Cosine"]    = data["Similarity_Score_Cosine"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Euclidean"] = data["Similarity_Score_Euclidean"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Manhattan"] = data["Similarity_Score_Manhattan"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Minkowski"] = data["Similarity_Score_Minkowski"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Pearson"]   = data["Similarity_Score_Pearson"].apply(lambda x: predict_1(x, 1))
data["predicted_labels_Canberra"]  = data["Similarity_Score_Canberra"].apply(lambda x: predict_1(x, 1))

---
### `Report` 

#### `Process:` Counts

In [9]:
# Count values 
count_Cosine    = data["predicted_labels_Cosine"].value_counts() 
count_Euclidean = data["predicted_labels_Euclidean"].value_counts() 
count_Manhattan = data["predicted_labels_Manhattan"].value_counts() 
count_Minkowski = data["predicted_labels_Minkowski"].value_counts()  
count_Pearson = data["predicted_labels_Pearson"].value_counts()  
count_Canberra = data["predicted_labels_Canberra"].value_counts()  
# Total
count_Combined  = pd.DataFrame({"Cosine": count_Cosine.values,       "Euclidean": count_Euclidean.values, 
                                "Manhattan": count_Manhattan.values, "Minkowski": count_Minkowski.values, 
                                "Pearson": count_Pearson,            "Canberra": count_Canberra}, index = list(count_Cosine.index))
## Add sum row to the end of the dataframe
count_Combined.loc['Total',:]= count_Combined.sum(axis=0) 

#### `Process:` Means

In [10]:
# Mean Values
mean_Cosine, mean_Euclidean, mean_Manhattan, mean_Minkowski, mean_Pearson, mean_Canberra = [], [], [], [], [], [] 
for label in current_Labels: 
    mean_Cosine.append( data["Similarity_Score_Cosine"].apply(lambda x: x[label]).mean() ) 
    mean_Euclidean.append( data["Similarity_Score_Euclidean"].apply(lambda x: x[label]).mean() ) 
    mean_Manhattan.append( data["Similarity_Score_Manhattan"].apply(lambda x: x[label]).mean() ) 
    mean_Minkowski.append( data["Similarity_Score_Minkowski"].apply(lambda x: x[label]).mean() ) 
    mean_Pearson.append( data["Similarity_Score_Pearson"].apply(lambda x: x[label]).mean() )  
    mean_Canberra.append( data["Similarity_Score_Canberra"].apply(lambda x: x[label]).mean() )  
# Combined 
mean_Combined = pd.DataFrame({"Cosine": mean_Cosine,       "Euclidean": mean_Euclidean, 
                              "Manhattan": mean_Manhattan, "Minkowski": mean_Minkowski, 
                              "Pearson": mean_Pearson,     "Canberra": mean_Canberra, 
                              }, index = list(current_Labels))
## Add sum row to the end of the dataframe
mean_Combined.loc['Total',:]= mean_Combined.mean(axis=0)

#### `Process:` Precision, Recall, F1, Support  

In [11]:
titles = create_Report("predicted_labels_Cosine")['title'].tolist() 

In [12]:
report_Cosine = create_Report("predicted_labels_Cosine")   
report_Cosine['title'] = "Cosine"
# Add total row to the end of the dataframe
temp = report_Cosine.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Cosine.loc[len(report_Cosine)] = temp  

In [13]:
report_Minkowski = create_Report("predicted_labels_Minkowski")   
report_Minkowski['title'] = "Minkowski"
# Add total row to the end of the dataframe
temp = report_Minkowski.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Minkowski.loc[len(report_Minkowski)] = temp

In [14]:
report_Manhattan = create_Report("predicted_labels_Manhattan")   
report_Manhattan['title'] = "Manhattan"
# Add total row to the end of the dataframe
temp = report_Manhattan.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Manhattan.loc[len(report_Manhattan)] = temp

In [15]:
report_Euclidean = create_Report("predicted_labels_Euclidean")   
report_Euclidean['title'] = "Euclidean"
# Add total row to the end of the dataframe
temp = report_Euclidean.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Euclidean.loc[len(report_Euclidean)] = temp

In [16]:
report_Pearson = create_Report("predicted_labels_Pearson")   
report_Pearson['title'] = "Pearson"
# Add total row to the end of the dataframe
temp = report_Pearson.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Pearson.loc[len(report_Pearson)] = temp

In [17]:
report_Canberra = create_Report("predicted_labels_Canberra")   
report_Canberra['title'] = "Canberra"
# Add total row to the end of the dataframe
temp = report_Canberra.iloc[:,1:].mean(axis=0) 
temp = ['Total'] + list (temp)  
# Add temp to the end of the dataframe as a row
report_Canberra.loc[len(report_Canberra)] = temp

In [18]:
top = pd.concat([report_Cosine, report_Minkowski], axis=1) 
bottom = pd.concat([report_Manhattan, report_Euclidean], axis=1) 
bottom2 = pd.concat([report_Pearson, report_Canberra], axis=1) 
combined = pd.concat([top, bottom], axis=0) 
combined = pd.concat([combined, bottom2], axis=0) 

#### `Process:` Accuracy, Macro Avg, Weighted Avg

In [19]:
titles_2 = extract_summary_metrics("predicted_labels_Cosine")['title'].tolist()  

In [20]:
report_2_Cosine = extract_summary_metrics("predicted_labels_Cosine") 
report_2_Cosine['title'] = "Cosine"
report_2_Minkowski = extract_summary_metrics("predicted_labels_Minkowski") 
report_2_Minkowski['title'] = 'Minkowski'
report_2_Manhattan = extract_summary_metrics("predicted_labels_Manhattan") 
report_2_Manhattan['title'] = 'Manhattan' 
report_2_Euclidean = extract_summary_metrics("predicted_labels_Euclidean") 
report_2_Euclidean['title'] = 'Euclidean' 
report_2_Pearson = extract_summary_metrics("predicted_labels_Pearson") 
report_2_Pearson['title'] = 'Pearson' 
report_2_Canberra = extract_summary_metrics("predicted_labels_Canberra") 
report_2_Canberra['title'] = 'Canberra' 


In [21]:
top = pd.concat([report_2_Cosine, report_2_Minkowski], axis=1) 
bottom = pd.concat([report_2_Manhattan, report_2_Euclidean], axis=1) 
bottom2 = pd.concat([report_2_Pearson, report_2_Canberra], axis=1) 
combined_2 = pd.concat([top, bottom], axis=0) 
combined_2 = pd.concat([combined_2, bottom2], axis=0) 
combined_2.columns = (['title'] + titles_2 + ['X']) * 2 

In [22]:
mean_Combined

,Cosine,Euclidean,Manhattan,Minkowski,Pearson,Canberra
Terrible,0.741443,0.581928,0.042511,0.801893,0.741251,0.001081
Mediocre,0.755169,0.588494,0.043936,0.804875,0.754977,0.001098
Fair,0.744693,0.583331,0.042888,0.802765,0.744482,0.001084
Great,0.766012,0.594022,0.044726,0.809388,0.765827,0.001102
Outstanding,0.756556,0.589468,0.044115,0.805934,0.756361,0.001100
Total,0.752775,0.587449,0.043635,0.804971,0.752580,0.001093


In [23]:
count_Combined

,Cosine,Euclidean,Manhattan,Minkowski,Pearson,Canberra
Great,173.0,173.0,152.0,227.0,173.0,85.0
Outstanding,128.0,128.0,143.0,104.0,128.0,168.0
Mediocre,115.0,115.0,143.0,88.0,115.0,122.0
Terrible,84.0,84.0,62.0,81.0,84.0,91.0
Total,500.0,500.0,500.0,500.0,500.0,466.0


In [24]:
combined

,title,precision,recall,f1,support,title,precision,recall,f1,support
0,Cosine,1.000,0.000,0.000,100.0,Minkowski,1.000,0.000,0.000,100.0
1,Cosine,0.300,0.520,0.380,100.0,Minkowski,0.360,0.810,0.500,100.0
2,Cosine,0.410,0.470,0.440,100.0,Minkowski,0.430,0.380,0.400,100.0
3,Cosine,0.620,0.800,0.700,100.0,Minkowski,0.770,0.620,0.690,100.0
4,Cosine,0.950,0.800,0.870,100.0,Minkowski,0.870,0.900,0.880,100.0
5,Total,0.656,0.518,0.478,100.0,Total,0.686,0.542,0.494,100.0
0,Manhattan,1.000,0.000,0.000,100.0,Euclidean,1.000,0.000,0.000,100.0
1,Manhattan,0.290,0.410,0.340,100.0,Euclidean,0.300,0.520,0.380,100.0
2,Manhattan,0.360,0.510,0.420,100.0,Euclidean,0.410,0.470,0.440,100.0
3,Manhattan,0.620,0.950,0.750,100.0,Euclidean,0.620,0.800,0.700,100.0


In [25]:
combined_2

,title,accuracy,macro avg,weighted avg,X,title,accuracy,macro avg,weighted avg,X
0,Cosine,NaN,NaN,NaN,NaN,Minkowski,NaN,NaN,NaN,NaN
1,Cosine,500.00,NaN,NaN,NaN,Minkowski,500.00,NaN,NaN,NaN
2,Cosine,0.66,0.52,0.48,500.0,Minkowski,0.68,0.54,0.49,500.0
0,Manhattan,NaN,NaN,NaN,NaN,Euclidean,NaN,NaN,NaN,NaN
1,Manhattan,500.00,NaN,NaN,NaN,Euclidean,500.00,NaN,NaN,NaN
2,Manhattan,0.65,0.49,0.45,500.0,Euclidean,0.66,0.52,0.48,500.0
0,Pearson,NaN,NaN,NaN,NaN,Canberra,NaN,NaN,NaN,NaN
1,Pearson,500.00,NaN,NaN,NaN,Canberra,500.00,NaN,NaN,NaN
2,Pearson,0.66,0.52,0.48,500.0,Canberra,0.53,0.55,0.52,500.0


---
### `Checkpoint` 

In [29]:
import dill

# Save the entire session
dill.dump_session('session_A.pkl')  
# Actialize the session
#dill.load_session('session.pkl') 